# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

In [1]:
#bank-marketing-ml-classifiers
#Overall, the dataset represents results from 17 campaigns done over a period of 2.5 years (May'2008 to Nov'2010) corresponding to a total of 79354 contacts

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, roc_curve, accuracy_score, ConfusionMatrixDisplay, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer, ColumnTransformer, make_column_selector

In [6]:
url = 'https://raw.githubusercontent.com/ashrivastav33/bank-marketing-ml-classifiers/refs/heads/main/data/bank-additional-full.csv'
bank_mkt_df = pd.read_csv(url, sep = ';')

In [7]:
bank_mkt_df .head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [8]:
#Validating data is loaded correctly in the bank_mkt_df dataframe
bank_mkt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [10]:
##Exploring statistical details for bank_mkt_df dataframe
bank_mkt_df.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,41188.00000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000
mean,40.02406,258.285010,2.567593,962.475454,0.172963,0.081886,93.575664,-40.502600,3.621291,5167.035911
std,10.42125,259.279249,2.770014,186.910907,0.494901,1.570960,0.578840,4.628198,1.734447,72.251528
min,17.00000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.00000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.00000,180.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.00000,319.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.00000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [11]:
# Looking for missing values
print(bank_mkt_df.isnull().sum())

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64


In [ ]:
# It looks like there is no null data but some of the categorical variables have 'unknown', for the
# purpose of modelling we'll keep these categories as 'unknown'

In [13]:
y = np.where(df['y']=='yes',1,0)
X = bank_mkt_df.drop(['y','duration'], axis=1)

In [14]:
# Let's identify counts of unknown for each of the columns
X[X[['job','marital','education','default','housing','loan']] == 'unknown'].count() / X.count()

,0
age,0.000000
job,0.008012
marital,0.001942
education,0.042027
default,0.208726
housing,0.024036
loan,0.024036
contact,0.000000
month,0.000000
day_of_week,0.000000


In [15]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  campaign        41188 non-null  int64  
 11  pdays           41188 non-null  int64  
 12  previous        41188 non-null  int64  
 13  poutcome        41188 non-null  object 
 14  emp.var.rate    41188 non-null  float64
 15  cons.price.idx  41188 non-null  float64
 16  cons.conf.idx   41188 non-null  float64
 17  euribor3m       41188 non-null 

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [22]:
# The business objective is to develop a predictive algorithm that identifies customers likely to
# accept the offer based on a number customer characteristics - this will allow to focus the
# limited resources of the organization / call center on customers most likely to accept the offer
# and have a benefit of reducing frustration (due to calls) from customers definitely won't

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [23]:
basic_transformer = ColumnTransformer([('scale', StandardScaler(),\
        make_column_selector(dtype_include=['int64', 'float64'])),\
        ('encode', OneHotEncoder(handle_unknown = 'ignore',drop='if_binary'),\
        make_column_selector(dtype_include=[object]))], sparse_threshold = 0)

In [30]:
X_transformed = pd.DataFrame(data = basic_transformer.fit_transform(X.iloc[:,:7]), columns = basic_transformer.get_feature_names_out())

#X_transformed = pd.DataFrame(data = basic_transformer.fit_transform(X), columns = basic_transformer.get_feature_names_out())

X_transformed.head()

,scale__age,encode__job_admin.,encode__job_blue-collar,encode__job_entrepreneur,encode__job_housemaid,encode__job_management,encode__job_retired,encode__job_self-employed,encode__job_services,encode__job_student,...,encode__education_unknown,encode__default_no,encode__default_unknown,encode__default_yes,encode__housing_no,encode__housing_unknown,encode__housing_yes,encode__loan_no,encode__loan_unknown,encode__loan_yes
0,1.533034,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,1.628993,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,-0.290186,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,-0.002309,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,1.533034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [31]:
df_for_corr = pd.concat([X_transformed,pd.DataFrame(y)], axis = 1)
df_for_corr.rename(columns = {0:'y'}, inplace = True)

In [32]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(
    go.Heatmap(
        x = df_for_corr.corr().columns,
        y = df_for_corr.corr().index,
        z = np.array(df_for_corr.corr()
        )
    )
)
fig.update_layout(title = 'Correlation Matrix')

In [33]:
# Let's identify parameters with highest correlation with y
df_for_corr.corr().sort_values('y', ascending=False)['y'].head(5)

,y
y,1.000000
encode__default_no,0.099344
encode__job_student,0.093955
encode__job_retired,0.092221
encode__marital_single,0.054133


### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size = .2, random_state = 32)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [35]:
# Baseline accuracy is equal to the frequency of the most common class (0)
1-y.sum()/len(y)

np.float64(0.8873458288821987)

### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [37]:
import time
start_time = time.time()
lr = LogisticRegression(max_iter = 1000).fit(X_train,y_train)
lr_elapsed_time = time.time() - start_time
print(f'{lr_elapsed_time:.2f}s elapsed during training')

0.57s elapsed during training


### Problem 9: Score the Model

What is the accuracy of your model?

In [40]:
acc_sc = accuracy_score(y_test, lr.predict(X_test))
print(acc_sc)

0.8833454722019908


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [41]:
start_time=time.time()
knn = KNeighborsClassifier().fit(X_train,y_train)
knn_elapsed_time = time.time() - start_time
print(f'{knn_elapsed_time:.2f}s elapsed during training')

0.01s elapsed during training


In [43]:
start_time=time.time()
dt = DecisionTreeClassifier().fit(X_train,y_train)
dt_elapsed_time = time.time() - start_time
print(f'{dt_elapsed_time:.2f}s elapsed during training')

0.28s elapsed during training


In [44]:
start_time=time.time()
svm = SVC().fit(X_train,y_train)
svm_elapsed_time = time.time() - start_time
print(f'{svm_elapsed_time:.2f}s elapsed during training')

127.22s elapsed during training


In [46]:
results = pd.DataFrame({'model':['Logistic Regression',' KNN', 'Decision Tree', 'SVM'],\
                       'Train Time':[lr_elapsed_time,knn_elapsed_time,dt_elapsed_time,svm_elapsed_time],\
                       'Train Accuracy': [accuracy_score(y_train, lr.predict(X_train)), accuracy_score(y_train,knn.predict(X_train)),\
                        accuracy_score(y_train, dt.predict(X_train)),accuracy_score(y_train, svm.predict(X_train))],\
                       'Test Accuracy':[accuracy_score(y_test, lr.predict(X_test)), accuracy_score(y_test,knn.predict(X_test)),\
                        accuracy_score(y_test, dt.predict(X_test)),accuracy_score(y_test, svm.predict(X_test))]})

In [47]:
results.set_index('model')

,Train Time,Train Accuracy,Test Accuracy
model,,,
Logistic Regression,0.570210,0.888346,0.883345
KNN,0.012277,0.891108,0.869750
Decision Tree,0.283890,0.917360,0.862952
SVM,127.219107,0.889378,0.882496


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.


- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

##### Questions